In [22]:
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime 
from datetime import timedelta
from datetime import date
# import datetime
import json
import os
from sys import getsizeof
# import datetime
import random
# from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
# from py_vollib.black_scholes.greeks import analytical as greeks
# import time
# import ks_api_client
from kiteconnect import KiteConnect
# from kiteconnect import KiteTicker

In [5]:
current_datetime = datetime(2021,5,17,15,0)
start_datetime = datetime(2021,5,17,15,15)
historical_data_folder_name = 'historical data'
fno_folder_name='FNO'
equity_folder_name="Equity"
underlying_name= 'NIFTY'

ltp_to_position_distance = .3


parent = os.path.dirname(os.getcwd())

historical_data_folder_path = os.path.join(\
        parent,historical_data_folder_name)

fno_data_folder_path = os.path.join(\
        historical_data_folder_path,fno_folder_name)

equity_data_folder_path = os.path.join(\
    historical_data_folder_path,equity_folder_name)

first_day_string = current_datetime.strftime("%Y-%m-%d")
first_fno_file_path = fno_file_paths = [\
        os.path.join(fno_data_folder_path,f) \
        for f in os.listdir(fno_data_folder_path) \
            if os.path.isfile(os.path.join(fno_data_folder_path,f)) \
                & (f.split(".")[0].split("_")[0] == first_day_string)\
                    ]

first_fno_df = pd.read_csv(first_fno_file_path[0])

end_datetime = pd.to_datetime(first_fno_df['expiry_date']).min() + timedelta (hours=15, minutes =30)

delta = timedelta(days=1)
start_date = current_datetime.date()
datestring_list = []
while start_date <= end_datetime.date():
    # print(start_date)
    datestring_list.append(start_date.strftime("%Y-%m-%d"))
    start_date += delta



fno_file_paths = [\
        os.path.join(fno_data_folder_path,f) \
        for f in os.listdir(fno_data_folder_path) \
            if os.path.isfile(os.path.join(fno_data_folder_path,f)) \
                & (f.split(".")[0].split("_")[0] in datestring_list)\
                    ]
fno_file_paths.sort()

equity_file_paths = [\
    os.path.join(equity_data_folder_path,f) \
    for f in os.listdir(equity_data_folder_path) \
        if os.path.isfile(os.path.join(equity_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0] in datestring_list)\
                ]
equity_file_paths.sort()


def size_and_rows(count, df):
    print(f"\t{count}\tSize: {round(getsizeof(df)/1_000_000,2)} MB, \t {round(df[df['underlying'].isnull()].shape[0]/1_000,0)}k rows, {len(df['underlying'].unique())}")

equity_book = pd.read_csv(equity_file_paths[0])
equity_book.rename(columns={'Ticker':'ticker',
                        'LTP':'ltp',
                        'BuyPrice':'buy_price',
                        'SellPrice':'sell_price'},
                        inplace=True)
equity_book = equity_book[['ticker','ltp','timestamp']]

equity_book['timestamp'] = pd.to_datetime(\
                equity_book['timestamp'])

start_datetime_ltp = equity_book[(equity_book['timestamp']==start_datetime)]['ltp'].iloc[0]
strike_high = start_datetime_ltp * (1+ltp_to_position_distance)
strike_low = start_datetime_ltp * (1-ltp_to_position_distance)

tick_book = pd.DataFrame()
for equity_file_path, fno_file_path in zip (equity_file_paths,fno_file_paths):
    print(fno_file_path)
    day_tick_book = pd.concat(map(pd.read_csv,[fno_file_path,equity_file_path]))

    size_and_rows(1,day_tick_book)

    day_tick_book.rename(columns={'Ticker':'ticker',
                            'LTP':'ltp',
                            'BuyPrice':'buy_price',
                            'SellPrice':'sell_price'},
                            inplace=True)
    size_and_rows(2,day_tick_book)

    day_tick_book = day_tick_book[['ticker','ltp','buy_price',\
                        'sell_price','underlying','strike','call_put',\
                        'expiry_date','timestamp']]
    size_and_rows(3,day_tick_book)

    day_tick_book = day_tick_book[\
                (day_tick_book['underlying']==underlying_name) | \
                (day_tick_book['underlying'].isnull())]
    size_and_rows(4,day_tick_book)

    day_tick_book['expiry_datetime'] = pd.to_datetime(\
                        day_tick_book['expiry_date']) + \
                            timedelta (hours=15, minutes =30)
    size_and_rows(5,day_tick_book)

    day_tick_book['timestamp'] = pd.to_datetime(\
                    day_tick_book['timestamp'])
    size_and_rows(6,day_tick_book)

    day_tick_book = day_tick_book[day_tick_book['timestamp']>=current_datetime]
    size_and_rows(7,day_tick_book)

    day_tick_book.sort_values(by=['ticker','timestamp'],inplace=True)
    size_and_rows(8,day_tick_book)

    day_tick_book.drop_duplicates(subset=['ticker','timestamp'],keep='first',inplace=True)
    size_and_rows(9,day_tick_book)

    day_tick_book = day_tick_book[(day_tick_book['expiry_datetime'] == day_tick_book['expiry_datetime'].min()) | \
                        (day_tick_book['expiry_datetime'].isnull())]
    size_and_rows(10,day_tick_book)

    day_tick_book = day_tick_book[((day_tick_book['strike']>=strike_low) & (day_tick_book['strike']<=strike_high)) |\
                    (day_tick_book['strike'].isnull())]
    size_and_rows(11,day_tick_book)

    tick_book = pd.concat([tick_book,day_tick_book])



/Users/hg/Space/Code/BatraTwoPeaksStrategy/historical data/FNO/2021-05-17.csv
	1	Size: 2622.72 MB, 	 27.0k rows, 2
	2	Size: 2622.72 MB, 	 27.0k rows, 2
	3	Size: 1849.65 MB, 	 27.0k rows, 2
	4	Size: 1849.65 MB, 	 27.0k rows, 2
	5	Size: 1887.36 MB, 	 27.0k rows, 2
	6	Size: 1566.82 MB, 	 27.0k rows, 2
	7	Size: 135.15 MB, 	 5.0k rows, 2
	8	Size: 135.15 MB, 	 5.0k rows, 2
	9	Size: 128.03 MB, 	 5.0k rows, 2
	10	Size: 43.62 MB, 	 5.0k rows, 2
	11	Size: 43.62 MB, 	 5.0k rows, 2
/Users/hg/Space/Code/BatraTwoPeaksStrategy/historical data/FNO/2021-05-18.csv
	1	Size: 2853.05 MB, 	 30.0k rows, 2
	2	Size: 2853.05 MB, 	 30.0k rows, 2
	3	Size: 2012.07 MB, 	 30.0k rows, 2
	4	Size: 2012.07 MB, 	 30.0k rows, 2
	5	Size: 2053.1 MB, 	 30.0k rows, 2
	6	Size: 1704.4 MB, 	 30.0k rows, 2
	7	Size: 1704.4 MB, 	 30.0k rows, 2
	8	Size: 1704.4 MB, 	 30.0k rows, 2
	9	Size: 1624.54 MB, 	 29.0k rows, 2
	10	Size: 517.7 MB, 	 29.0k rows, 2
	11	Size: 517.7 MB, 	 29.0k rows, 2
/Users/hg/Space/Code/BatraTwoPeaksStrategy/his

In [7]:
instruments_book = tick_book[['ticker','underlying',\
                    'strike','call_put','expiry_datetime']]
instruments_book.drop_duplicates(inplace=True)
instruments_book

/var/folders/rw/d270qq7d70j5rl2p2xpt2yfr0000gn/T/ipykernel_1297/1353309311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instruments_book.drop_duplicates(inplace=True)


,ticker,underlying,strike,call_put,expiry_datetime
21560,NIFTY 50.NSE_IDX,NaN,NaN,NaN,NaT
208357,NIFTY20MAY2112800PE.NFO,NIFTY,12800.0,PE,2021-05-20 15:30:00
1157351,NIFTY20MAY2112850PE.NFO,NIFTY,12850.0,PE,2021-05-20 15:30:00
3295311,NIFTY20MAY2112900PE.NFO,NIFTY,12900.0,PE,2021-05-20 15:30:00
2454511,NIFTY20MAY2112950PE.NFO,NIFTY,12950.0,PE,2021-05-20 15:30:00
...,...,...,...,...,...
1088411,NIFTY20MAY2113700CE.NFO,NIFTY,13700.0,CE,2021-05-20 15:30:00
1167143,NIFTY20MAY2115850PE.NFO,NIFTY,15850.0,PE,2021-05-20 15:30:00
367981,NIFTY20MAY2116500CE.NFO,NIFTY,16500.0,CE,2021-05-20 15:30:00
623757,NIFTY20MAY2116550CE.NFO,NIFTY,16550.0,CE,2021-05-20 15:30:00


In [59]:
f = pd.DataFrame({"underlying":{"0":"NIFTY","1":"NIFTY"},"call_put":{"0":"CE","1":"PE"},"expiry_datetime":{"0":1621524600000,"1":1621524600000},"strike":{"0":19350,"1":10450}})
i = pd.DataFrame({"ticker":{"471":"NIFTY 50.NSE_IDX","206610":"NIFTY20MAY2112800PE.NFO","1157237":"NIFTY20MAY2112850PE.NFO","3294358":"NIFTY20MAY2112900PE.NFO","2453921":"NIFTY20MAY2112950PE.NFO","3699629":"NIFTY20MAY2113000CE.NFO","2525438":"NIFTY20MAY2113000PE.NFO","3495615":"NIFTY20MAY2113050PE.NFO","907061":"NIFTY20MAY2113100PE.NFO","123574":"NIFTY20MAY2113150PE.NFO","1409029":"NIFTY20MAY2113200PE.NFO","2192220":"NIFTY20MAY2113250PE.NFO","4593124":"NIFTY20MAY2113300PE.NFO","3589343":"NIFTY20MAY2113350PE.NFO","161426":"NIFTY20MAY2113400PE.NFO","973413":"NIFTY20MAY2113450PE.NFO","4415563":"NIFTY20MAY2113500CE.NFO","3455115":"NIFTY20MAY2113500PE.NFO","2454579":"NIFTY20MAY2113550PE.NFO","3519618":"NIFTY20MAY2113600PE.NFO","4554673":"NIFTY20MAY2113650PE.NFO","2241712":"NIFTY20MAY2113700PE.NFO","1425026":"NIFTY20MAY2113750PE.NFO","2069228":"NIFTY20MAY2113800CE.NFO","582990":"NIFTY20MAY2113800PE.NFO","1670773":"NIFTY20MAY2113850CE.NFO","400897":"NIFTY20MAY2113850PE.NFO","3854172":"NIFTY20MAY2113900CE.NFO","2877003":"NIFTY20MAY2113900PE.NFO","3203584":"NIFTY20MAY2113950PE.NFO","3628230":"NIFTY20MAY2114000CE.NFO","2457986":"NIFTY20MAY2114000PE.NFO","4384357":"NIFTY20MAY2114050CE.NFO","3437373":"NIFTY20MAY2114050PE.NFO","2343637":"NIFTY20MAY2114100CE.NFO","954236":"NIFTY20MAY2114100PE.NFO","1358435":"NIFTY20MAY2114150CE.NFO","170348":"NIFTY20MAY2114150PE.NFO","50459":"NIFTY20MAY2114200CE.NFO","1430434":"NIFTY20MAY2114200PE.NFO","1051096":"NIFTY20MAY2114250CE.NFO","2223330":"NIFTY20MAY2114250PE.NFO","3348580":"NIFTY20MAY2114300CE.NFO","4526905":"NIFTY20MAY2114300PE.NFO","2556904":"NIFTY20MAY2114350CE.NFO","3529545":"NIFTY20MAY2114350PE.NFO","1265628":"NIFTY20MAY2114400CE.NFO","101126":"NIFTY20MAY2114400PE.NFO","2293117":"NIFTY20MAY2114450CE.NFO","914606":"NIFTY20MAY2114450PE.NFO","4484317":"NIFTY20MAY2114500CE.NFO","3497051":"NIFTY20MAY2114500PE.NFO","3677855":"NIFTY20MAY2114550CE.NFO","2503074":"NIFTY20MAY2114550PE.NFO","2600186":"NIFTY20MAY2114600CE.NFO","3566767":"NIFTY20MAY2114600PE.NFO","3404330":"NIFTY20MAY2114650CE.NFO","4594914":"NIFTY20MAY2114650PE.NFO","1015641":"NIFTY20MAY2114700CE.NFO","2192912":"NIFTY20MAY2114700PE.NFO","0":"NIFTY20MAY2114750CE.NFO","1384294":"NIFTY20MAY2114750PE.NFO","2046200":"NIFTY20MAY2114800CE.NFO","560365":"NIFTY20MAY2114800PE.NFO","1636407":"NIFTY20MAY2114850CE.NFO","370373":"NIFTY20MAY2114850PE.NFO","3879216":"NIFTY20MAY2114900CE.NFO","2900971":"NIFTY20MAY2114900PE.NFO","4142914":"NIFTY20MAY2114950CE.NFO","3225677":"NIFTY20MAY2114950PE.NFO","4342947":"NIFTY20MAY2115000CE.NFO","3034190":"NIFTY20MAY2115000PE.NFO","3963669":"NIFTY20MAY2115050CE.NFO","2821541":"NIFTY20MAY2115050PE.NFO","1551705":"NIFTY20MAY2115100CE.NFO","410622":"NIFTY20MAY2115100PE.NFO","1774791":"NIFTY20MAY2115150CE.NFO","825930":"NIFTY20MAY2115150PE.NFO","714020":"NIFTY20MAY2115200CE.NFO","1893405":"NIFTY20MAY2115200PE.NFO","494034":"NIFTY20MAY2115250CE.NFO","1459754":"NIFTY20MAY2115250PE.NFO","2691803":"NIFTY20MAY2115300CE.NFO","4048962":"NIFTY20MAY2115300PE.NFO","3124451":"NIFTY20MAY2115350CE.NFO","4271415":"NIFTY20MAY2115350PE.NFO","1823782":"NIFTY20MAY2115400CE.NFO","866224":"NIFTY20MAY2115400PE.NFO","1620623":"NIFTY20MAY2115450CE.NFO","480085":"NIFTY20MAY2115450PE.NFO","3920442":"NIFTY20MAY2115500CE.NFO","2778613":"NIFTY20MAY2115500PE.NFO","4297790":"NIFTY20MAY2115550CE.NFO","2980301":"NIFTY20MAY2115550PE.NFO","3096332":"NIFTY20MAY2115600CE.NFO","2670870":"NIFTY20MAY2115650CE.NFO","522431":"NIFTY20MAY2115700CE.NFO","749729":"NIFTY20MAY2115750CE.NFO","1925240":"NIFTY20MAY2115750PE.NFO","1257112":"NIFTY20MAY2115800CE.NFO","226314":"NIFTY20MAY2115800PE.NFO","2100761":"NIFTY20MAY2115850CE.NFO","4660254":"NIFTY20MAY2115900CE.NFO","3250658":"NIFTY20MAY2115900PE.NFO","3746119":"NIFTY20MAY2115950CE.NFO","597939":"NIFTY20MAY2116000CE.NFO","2079453":"NIFTY20MAY2116000PE.NFO","393804":"NIFTY20MAY2116050CE.NFO","2875662":"NIFTY20MAY2116100CE.NFO","3217914":"NIFTY20MAY2116150CE.NFO","4177686":"NIFTY20MAY2116200CE.NFO","3816411":"NIFTY20MAY2116250CE.NFO","1720682":"NIFTY20MAY2116300CE.NFO","2023476":"NIFTY20MAY2116350CE.NFO","3248414":"NIFTY20MAY2116400CE.NFO","2899660":"NIFTY20MAY2116450CE.NFO","25137":"NIFTY20MAY2113200CE.NFO","4465610":"NIFTY20MAY2113950CE.NFO","4580032":"NIFTY20MAY2115600PE.NFO","4351934":"NIFTY20MAY2115650PE.NFO","1620919":"NIFTY20MAY2115700PE.NFO","2632492":"NIFTY20MAY2115950PE.NFO","4169845":"NIFTY20MAY2116100PE.NFO","1088411":"NIFTY20MAY2113700CE.NFO","1167143":"NIFTY20MAY2115850PE.NFO","367981":"NIFTY20MAY2116500CE.NFO","623757":"NIFTY20MAY2116550CE.NFO","2065656":"NIFTY20MAY2116600CE.NFO","1198138":"NIFTY20MAY2112800CE.NFO","3786548":"NIFTY20MAY2116450PE.NFO"},"underlying":{"471":None,"206610":"NIFTY","1157237":"NIFTY","3294358":"NIFTY","2453921":"NIFTY","3699629":"NIFTY","2525438":"NIFTY","3495615":"NIFTY","907061":"NIFTY","123574":"NIFTY","1409029":"NIFTY","2192220":"NIFTY","4593124":"NIFTY","3589343":"NIFTY","161426":"NIFTY","973413":"NIFTY","4415563":"NIFTY","3455115":"NIFTY","2454579":"NIFTY","3519618":"NIFTY","4554673":"NIFTY","2241712":"NIFTY","1425026":"NIFTY","2069228":"NIFTY","582990":"NIFTY","1670773":"NIFTY","400897":"NIFTY","3854172":"NIFTY","2877003":"NIFTY","3203584":"NIFTY","3628230":"NIFTY","2457986":"NIFTY","4384357":"NIFTY","3437373":"NIFTY","2343637":"NIFTY","954236":"NIFTY","1358435":"NIFTY","170348":"NIFTY","50459":"NIFTY","1430434":"NIFTY","1051096":"NIFTY","2223330":"NIFTY","3348580":"NIFTY","4526905":"NIFTY","2556904":"NIFTY","3529545":"NIFTY","1265628":"NIFTY","101126":"NIFTY","2293117":"NIFTY","914606":"NIFTY","4484317":"NIFTY","3497051":"NIFTY","3677855":"NIFTY","2503074":"NIFTY","2600186":"NIFTY","3566767":"NIFTY","3404330":"NIFTY","4594914":"NIFTY","1015641":"NIFTY","2192912":"NIFTY","0":"NIFTY","1384294":"NIFTY","2046200":"NIFTY","560365":"NIFTY","1636407":"NIFTY","370373":"NIFTY","3879216":"NIFTY","2900971":"NIFTY","4142914":"NIFTY","3225677":"NIFTY","4342947":"NIFTY","3034190":"NIFTY","3963669":"NIFTY","2821541":"NIFTY","1551705":"NIFTY","410622":"NIFTY","1774791":"NIFTY","825930":"NIFTY","714020":"NIFTY","1893405":"NIFTY","494034":"NIFTY","1459754":"NIFTY","2691803":"NIFTY","4048962":"NIFTY","3124451":"NIFTY","4271415":"NIFTY","1823782":"NIFTY","866224":"NIFTY","1620623":"NIFTY","480085":"NIFTY","3920442":"NIFTY","2778613":"NIFTY","4297790":"NIFTY","2980301":"NIFTY","3096332":"NIFTY","2670870":"NIFTY","522431":"NIFTY","749729":"NIFTY","1925240":"NIFTY","1257112":"NIFTY","226314":"NIFTY","2100761":"NIFTY","4660254":"NIFTY","3250658":"NIFTY","3746119":"NIFTY","597939":"NIFTY","2079453":"NIFTY","393804":"NIFTY","2875662":"NIFTY","3217914":"NIFTY","4177686":"NIFTY","3816411":"NIFTY","1720682":"NIFTY","2023476":"NIFTY","3248414":"NIFTY","2899660":"NIFTY","25137":"NIFTY","4465610":"NIFTY","4580032":"NIFTY","4351934":"NIFTY","1620919":"NIFTY","2632492":"NIFTY","4169845":"NIFTY","1088411":"NIFTY","1167143":"NIFTY","367981":"NIFTY","623757":"NIFTY","2065656":"NIFTY","1198138":"NIFTY","3786548":"NIFTY"},"strike":{"471":None,"206610":12800.0,"1157237":12850.0,"3294358":12900.0,"2453921":12950.0,"3699629":13000.0,"2525438":13000.0,"3495615":13050.0,"907061":13100.0,"123574":13150.0,"1409029":13200.0,"2192220":13250.0,"4593124":13300.0,"3589343":13350.0,"161426":13400.0,"973413":13450.0,"4415563":13500.0,"3455115":13500.0,"2454579":13550.0,"3519618":13600.0,"4554673":13650.0,"2241712":13700.0,"1425026":13750.0,"2069228":13800.0,"582990":13800.0,"1670773":13850.0,"400897":13850.0,"3854172":13900.0,"2877003":13900.0,"3203584":13950.0,"3628230":14000.0,"2457986":14000.0,"4384357":14050.0,"3437373":14050.0,"2343637":14100.0,"954236":14100.0,"1358435":14150.0,"170348":14150.0,"50459":14200.0,"1430434":14200.0,"1051096":14250.0,"2223330":14250.0,"3348580":14300.0,"4526905":14300.0,"2556904":14350.0,"3529545":14350.0,"1265628":14400.0,"101126":14400.0,"2293117":14450.0,"914606":14450.0,"4484317":14500.0,"3497051":14500.0,"3677855":14550.0,"2503074":14550.0,"2600186":14600.0,"3566767":14600.0,"3404330":14650.0,"4594914":14650.0,"1015641":14700.0,"2192912":14700.0,"0":14750.0,"1384294":14750.0,"2046200":14800.0,"560365":14800.0,"1636407":14850.0,"370373":14850.0,"3879216":14900.0,"2900971":14900.0,"4142914":14950.0,"3225677":14950.0,"4342947":15000.0,"3034190":15000.0,"3963669":15050.0,"2821541":15050.0,"1551705":15100.0,"410622":15100.0,"1774791":15150.0,"825930":15150.0,"714020":15200.0,"1893405":15200.0,"494034":15250.0,"1459754":15250.0,"2691803":15300.0,"4048962":15300.0,"3124451":15350.0,"4271415":15350.0,"1823782":15400.0,"866224":15400.0,"1620623":15450.0,"480085":15450.0,"3920442":15500.0,"2778613":15500.0,"4297790":15550.0,"2980301":15550.0,"3096332":15600.0,"2670870":15650.0,"522431":15700.0,"749729":15750.0,"1925240":15750.0,"1257112":15800.0,"226314":15800.0,"2100761":15850.0,"4660254":15900.0,"3250658":15900.0,"3746119":15950.0,"597939":16000.0,"2079453":16000.0,"393804":16050.0,"2875662":16100.0,"3217914":16150.0,"4177686":16200.0,"3816411":16250.0,"1720682":16300.0,"2023476":16350.0,"3248414":16400.0,"2899660":16450.0,"25137":13200.0,"4465610":13950.0,"4580032":15600.0,"4351934":15650.0,"1620919":15700.0,"2632492":15950.0,"4169845":16100.0,"1088411":13700.0,"1167143":15850.0,"367981":16500.0,"623757":16550.0,"2065656":16600.0,"1198138":12800.0,"3786548":16450.0},"call_put":{"471":None,"206610":"PE","1157237":"PE","3294358":"PE","2453921":"PE","3699629":"CE","2525438":"PE","3495615":"PE","907061":"PE","123574":"PE","1409029":"PE","2192220":"PE","4593124":"PE","3589343":"PE","161426":"PE","973413":"PE","4415563":"CE","3455115":"PE","2454579":"PE","3519618":"PE","4554673":"PE","2241712":"PE","1425026":"PE","2069228":"CE","582990":"PE","1670773":"CE","400897":"PE","3854172":"CE","2877003":"PE","3203584":"PE","3628230":"CE","2457986":"PE","4384357":"CE","3437373":"PE","2343637":"CE","954236":"PE","1358435":"CE","170348":"PE","50459":"CE","1430434":"PE","1051096":"CE","2223330":"PE","3348580":"CE","4526905":"PE","2556904":"CE","3529545":"PE","1265628":"CE","101126":"PE","2293117":"CE","914606":"PE","4484317":"CE","3497051":"PE","3677855":"CE","2503074":"PE","2600186":"CE","3566767":"PE","3404330":"CE","4594914":"PE","1015641":"CE","2192912":"PE","0":"CE","1384294":"PE","2046200":"CE","560365":"PE","1636407":"CE","370373":"PE","3879216":"CE","2900971":"PE","4142914":"CE","3225677":"PE","4342947":"CE","3034190":"PE","3963669":"CE","2821541":"PE","1551705":"CE","410622":"PE","1774791":"CE","825930":"PE","714020":"CE","1893405":"PE","494034":"CE","1459754":"PE","2691803":"CE","4048962":"PE","3124451":"CE","4271415":"PE","1823782":"CE","866224":"PE","1620623":"CE","480085":"PE","3920442":"CE","2778613":"PE","4297790":"CE","2980301":"PE","3096332":"CE","2670870":"CE","522431":"CE","749729":"CE","1925240":"PE","1257112":"CE","226314":"PE","2100761":"CE","4660254":"CE","3250658":"PE","3746119":"CE","597939":"CE","2079453":"PE","393804":"CE","2875662":"CE","3217914":"CE","4177686":"CE","3816411":"CE","1720682":"CE","2023476":"CE","3248414":"CE","2899660":"CE","25137":"CE","4465610":"CE","4580032":"PE","4351934":"PE","1620919":"PE","2632492":"PE","4169845":"PE","1088411":"CE","1167143":"PE","367981":"CE","623757":"CE","2065656":"CE","1198138":"CE","3786548":"PE"},"expiry_datetime":{"471":None,"206610":1621524600000,"1157237":1621524600000,"3294358":1621524600000,"2453921":1621524600000,"3699629":1621524600000,"2525438":1621524600000,"3495615":1621524600000,"907061":1621524600000,"123574":1621524600000,"1409029":1621524600000,"2192220":1621524600000,"4593124":1621524600000,"3589343":1621524600000,"161426":1621524600000,"973413":1621524600000,"4415563":1621524600000,"3455115":1621524600000,"2454579":1621524600000,"3519618":1621524600000,"4554673":1621524600000,"2241712":1621524600000,"1425026":1621524600000,"2069228":1621524600000,"582990":1621524600000,"1670773":1621524600000,"400897":1621524600000,"3854172":1621524600000,"2877003":1621524600000,"3203584":1621524600000,"3628230":1621524600000,"2457986":1621524600000,"4384357":1621524600000,"3437373":1621524600000,"2343637":1621524600000,"954236":1621524600000,"1358435":1621524600000,"170348":1621524600000,"50459":1621524600000,"1430434":1621524600000,"1051096":1621524600000,"2223330":1621524600000,"3348580":1621524600000,"4526905":1621524600000,"2556904":1621524600000,"3529545":1621524600000,"1265628":1621524600000,"101126":1621524600000,"2293117":1621524600000,"914606":1621524600000,"4484317":1621524600000,"3497051":1621524600000,"3677855":1621524600000,"2503074":1621524600000,"2600186":1621524600000,"3566767":1621524600000,"3404330":1621524600000,"4594914":1621524600000,"1015641":1621524600000,"2192912":1621524600000,"0":1621524600000,"1384294":1621524600000,"2046200":1621524600000,"560365":1621524600000,"1636407":1621524600000,"370373":1621524600000,"3879216":1621524600000,"2900971":1621524600000,"4142914":1621524600000,"3225677":1621524600000,"4342947":1621524600000,"3034190":1621524600000,"3963669":1621524600000,"2821541":1621524600000,"1551705":1621524600000,"410622":1621524600000,"1774791":1621524600000,"825930":1621524600000,"714020":1621524600000,"1893405":1621524600000,"494034":1621524600000,"1459754":1621524600000,"2691803":1621524600000,"4048962":1621524600000,"3124451":1621524600000,"4271415":1621524600000,"1823782":1621524600000,"866224":1621524600000,"1620623":1621524600000,"480085":1621524600000,"3920442":1621524600000,"2778613":1621524600000,"4297790":1621524600000,"2980301":1621524600000,"3096332":1621524600000,"2670870":1621524600000,"522431":1621524600000,"749729":1621524600000,"1925240":1621524600000,"1257112":1621524600000,"226314":1621524600000,"2100761":1621524600000,"4660254":1621524600000,"3250658":1621524600000,"3746119":1621524600000,"597939":1621524600000,"2079453":1621524600000,"393804":1621524600000,"2875662":1621524600000,"3217914":1621524600000,"4177686":1621524600000,"3816411":1621524600000,"1720682":1621524600000,"2023476":1621524600000,"3248414":1621524600000,"2899660":1621524600000,"25137":1621524600000,"4465610":1621524600000,"4580032":1621524600000,"4351934":1621524600000,"1620919":1621524600000,"2632492":1621524600000,"4169845":1621524600000,"1088411":1621524600000,"1167143":1621524600000,"367981":1621524600000,"623757":1621524600000,"2065656":1621524600000,"1198138":1621524600000,"3786548":1621524600000}})
i

,ticker,underlying,strike,call_put,expiry_datetime
471,NIFTY 50.NSE_IDX,None,NaN,None,NaN
206610,NIFTY20MAY2112800PE.NFO,NIFTY,12800.0,PE,1.621525e+12
1157237,NIFTY20MAY2112850PE.NFO,NIFTY,12850.0,PE,1.621525e+12
3294358,NIFTY20MAY2112900PE.NFO,NIFTY,12900.0,PE,1.621525e+12
2453921,NIFTY20MAY2112950PE.NFO,NIFTY,12950.0,PE,1.621525e+12
...,...,...,...,...,...
367981,NIFTY20MAY2116500CE.NFO,NIFTY,16500.0,CE,1.621525e+12
623757,NIFTY20MAY2116550CE.NFO,NIFTY,16550.0,CE,1.621525e+12
2065656,NIFTY20MAY2116600CE.NFO,NIFTY,16600.0,CE,1.621525e+12
1198138,NIFTY20MAY2112800CE.NFO,NIFTY,12800.0,CE,1.621525e+12


In [60]:
f.merge(i,how='left',\
                    left_on=['underlying','call_put','expiry_datetime','strike'], 
                    right_on=['underlying','call_put','expiry_datetime','strike'])\
                    ['ticker'].astype(str)

0    nan
1    nan
Name: ticker, dtype: object

In [64]:
"{:.0f}".format(30)

'30'

In [62]:
f.merge(i,how='left',\
                    left_on=['strike'], 
                    right_on=['strike'])\
                    ['ticker'].astype(str)

0    nan
1    nan
Name: ticker, dtype: object

In [42]:
d - nt

TypeError: unsupported operand type(s) for -: 'datetime.date' and 'datetime.datetime'